In [ ]:
# Working Example for Animation, used for transferring to Main-PathPoint
%matplotlib qt5
#matplotlib_animation
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from time import sleep
from threading import Thread

countryList = ["USA", "ZAF", "JPN"]
sampleDic = {'2020-10-03': {'USA': 48674, 'ZAF': 1883, 'JPN': 577}, '2020-10-04': {'USA': 36092, 'ZAF': 1573, 'JPN': 401}, '2020-10-05': {'USA': 38917, 'ZAF': 926, 'JPN': 281}, '2020-10-06': {'USA': 43949, 'ZAF': 1027, 'JPN': 513}, '2020-10-07': {'USA': 50641, 'ZAF': 1913, 'JPN': 499}, '2020-10-08': {'USA': 58329, 'ZAF': 1736, 'JPN': 640}, '2020-10-09': {'USA': 55994, 'ZAF': 1461, 'JPN': 588}, '2020-10-10': {'USA': 54007, 'ZAF': 2544, 'JPN': 695}, '2020-10-11': {'USA': 45834, 'ZAF': 1575, 'JPN': 438}, '2020-10-12': {'USA': 40704, 'ZAF': 888, 'JPN': 252}, '2020-10-13': {'USA': 52879, 'ZAF': 1178, 'JPN': 501}, '2020-10-14': {'USA': 59473, 'ZAF': 1877, 'JPN': 541}, '2020-10-15': {'USA': 64358, 'ZAF': 1770, 'JPN': 708}, '2020-10-16': {'USA': 68449, 'ZAF': 2019, 'JPN': 642}, '2020-10-17': {'USA': 56875, 'ZAF': 1928, 'JPN': 626}, '2020-10-18': {'USA': 49296, 'ZAF': 1662, 'JPN': 428}, '2020-10-19': {'USA': 67006, 'ZAF': 1461, 'JPN': 310}, '2020-10-20': {'USA': 61088, 'ZAF': 1050, 'JPN': 487}, '2020-10-21': {'USA': 63026, 'ZAF': 2055, 'JPN': 620}, '2020-10-22': {'USA': 75335, 'ZAF': 2156, 'JPN': 619}, '2020-10-23': {'USA': 80987, 'ZAF': 1897, 'JPN': 734}, '2020-10-24': {'USA': 82787, 'ZAF': 1834, 'JPN': 731}, '2020-10-25': {'USA': 61761, 'ZAF': 1622, 'JPN': 496}, '2020-10-26': {'USA': 66579, 'ZAF': 891, 'JPN': 408}, '2020-10-27': {'USA': 76001, 'ZAF': 1092, 'JPN': 643}, '2020-10-28': {'USA': 78684, 'ZAF': 1863, 'JPN': 731}, '2020-10-29': {'USA': 90168, 'ZAF': 2056, 'JPN': 797}, '2020-10-30': {'USA': 98519, 'ZAF': 1912, 'JPN': 776}, '2020-10-31': {'USA': 88884, 'ZAF': 1770, 'JPN': 877}, '2020-11-01': {'USA': 75562, 'ZAF': 1371, 'JPN': 616}, '2020-11-02': {'USA': 83484, 'ZAF': 772, 'JPN': 488}, '2020-11-03': {'USA': 124618, 'ZAF': 1241, 'JPN': 878}, '2020-11-04': {'USA': 102873, 'ZAF': 1712, 'JPN': 619}, '2020-11-05': {'USA': 126190, 'ZAF': 1866, 'JPN': 1036}, '2020-11-06': {'USA': 125708, 'ZAF': 1761, 'JPN': 1172}, '2020-11-07': {'USA': 128000, 'ZAF': 1731, 'JPN': 1303}, '2020-11-08': {'USA': 109906, 'ZAF': 1372, 'JPN': 955}, '2020-11-09': {'USA': 121366, 'ZAF': 1247, 'JPN': 797}, '2020-11-10': {'USA': 137694, 'ZAF': 1729, 'JPN': 1296}, '2020-11-11': {'USA': 143548, 'ZAF': 2140, 'JPN': 1524}, '2020-11-12': {'USA': 161248, 'ZAF': 2338, 'JPN': 1644}, '2020-11-13': {'USA': 177568, 'ZAF': 2213, 'JPN': 1705}, '2020-11-14': {'USA': 166709, 'ZAF': 2237, 'JPN': 1753}, '2020-11-15': {'USA': 135785, 'ZAF': 1842, 'JPN': 1498}, '2020-11-16': {'USA': 157766, 'ZAF': 1245, 'JPN': 946}, '2020-11-17': {'USA': 160515, 'ZAF': 1987, 'JPN': 1690}, '2020-11-18': {'USA': 170401, 'ZAF': 2888, 'JPN': 2230}, '2020-11-19': {'USA': 187963, 'ZAF': 2514, 'JPN': 2382}, '2020-11-20': {'USA': 196010, 'ZAF': 3105, 'JPN': 2426}, '2020-11-21': {'USA': 178073, 'ZAF': 2646, 'JPN': 2586}, '2020-11-22': {'USA': 142734, 'ZAF': 2270, 'JPN': 2163}, '2020-11-23': {'USA': 171462, 'ZAF': 2080, 'JPN': 1520}, '2020-11-24': {'USA': 172935, 'ZAF': 2493, 'JPN': 1232}, '2020-11-25': {'USA': 181490, 'ZAF': 3250, 'JPN': 1949}, '2020-11-26': {'USA': 110611, 'ZAF': 3069, 'JPN': 2490}, '2020-11-27': {'USA': 205557, 'ZAF': 3370, 'JPN': 2553}, '2020-11-28': {'USA': 155866, 'ZAF': 3198, 'JPN': 2679}, '2020-11-29': {'USA': 138633, 'ZAF': 2563, 'JPN': 2058}, '2020-11-30': {'USA': 157901, 'ZAF': 2302, 'JPN': 1447}, '2020-12-01': {'USA': 180083, 'ZAF': 2295, 'JPN': 2014}, '2020-12-02': {'USA': 200070, 'ZAF': 4173, 'JPN': 2456}}

# coord_Dic ={'USA': [0.325, 0.79], 'ZAF': [0.708, 0.431], 'JPN': [0.348, 0.102]}
coord_Dic ={'USA': [156, 552], 'ZAF': [340, 301], 'JPN': [167, 71]}
maxCountryCases= {'USA': 205557, 'ZAF': 13944, 'JPN': 2679}

fig, ax = plt.subplots(figsize=(6,9))


img = plt.imread("CSVData\world3.jpg")
ax.imshow(img)

area = [205557, 3370, 2553]

countryPercentDisplay = []


# plt.show()
modifier = 30
color = {"USA":'tab:red', "ZAF":'tab:purple', "JPN":'tab:green'}

plt.show()

for date,country_data in sampleDic.items():
    # items = '2020-11-27': {'USA': 205557, 'ZAF': 3370, 'JPN': 2553}
    newArea = []
    caseDisplay = []    

    for country, cases in country_data.items():
        # country_data = {'USA': 205557, 'ZAF': 3370, 'JPN': 2553}
        percValue = round(cases / maxCountryCases[country] * 100)
        countryPercentDisplay.append({country: percValue})
        # newArea.append(percValue*30)
        caseDisplay.append(cases)   # For Text Display
        # ======== Matplotlib ========
        ax.scatter(
                coord_Dic[country][0],  #Get X Coordinate of Country
                coord_Dic[country][1],  #Get Y Coordinate of Country
                s=percValue*modifier,   #Area based on Max percent*(Modifier)
                c=color[country],
                label=country,
                alpha=0.6
        )

    # ======== Matplotlib ========
    area = newArea
    
    ax.imshow(img)
    ax.grid(True)

    ax.set_title(f"COVID New Cases Worldwide: Date: {date}",loc="left")

    dataText = f"""
    New Cases:
    United States:  {caseDisplay[0]}
    South Africa:   {caseDisplay[1]}
    Japan:      {caseDisplay[2]}"""

    # ax.text(0,0,dataText,horizontalalignment="left")
    # ax.text(5,794,dataText)
    ax.text(100,790,dataText,horizontalalignment="center")
    # ax.text(0,0,dataText)

    # Adjust Legend Size
    lgnd = ax.legend(loc="lower right", scatterpoints=1, fontsize=10)
    for i in range(len(countryList)):
        lgnd.legendHandles[i]._sizes = [100]
    
    fig.canvas.draw()
    fig.canvas.flush_events()

# plt.show()


In [11]:
# =================================================================================
%matplotlib qt5
import cv2
import numpy as np
import sqlite3
from bhaptics import haptic_player
from time import sleep
# from threading import Thread
import threading
                            

pathPointList = []
countryPathPointCoordDic = {}       # For use in PathPoint (bHaptic Json)
# Example Output: {'USA': [156, 552], 'ZAF': [340, 301], 'JPN': [167, 71]}
coord_Dic = {}
# tempList = [("USA", 160, 539), ("JPN", 185, 306), ("BRA", 315, 260)]
countryList = ["USA", "ZAF", "JPN"]
sampleData = ['USA', '2020-01-23', '0']
maxCountryCases = {}  # USA, JPN, ZAF

# imageSize = [480, 699] #world2.jpg
imageSize = [550, 800] #world3.jpg
imageDir = "CSVData/world3.jpg"
imageDir_CV = "CSVData/world3+haptic.jpg"   #TODO: runCV2 imread
# imageDir = "CSVData/world3+haptic.jpg"

# ======= Main Data & Parameters =======
CoVID_Data = {}
selection = "date, iso_code, new_cases"
debugSkip = 300   #0 = Normal Run
guiCircleModifier = 30
color = {"USA":'tab:red', "ZAF":'tab:purple', "JPN":'tab:green'}

wait = 0.2          #Determine Sleep time after haptic (match "millis" for no delay)
millis = 300        #Length of Vibration per "submit_path" to BHaptic Player

# Testing Only
dateList = []
intensityList = {"USA": [], "ZAF": [], "JPN": []}

displayText = ""


def initializeCovidWorldData():
    # ('USA', '2020-01-23', '0')
    conn = sqlite3.connect('covid-world.db')
    c = conn.cursor()

    # Get
    def getMaxCountryCases(countryIsoCode):
        str = f"""SELECT MAX(new_cases) FROM "covid-world" WHERE "iso_code" LIKE '%{countryIsoCode}%'"""
        c.execute(str)
        num = c.fetchone()
        # print(f'num={num} | type={type(num)}')
        return int(num[0])

    def generateCoVIDData(countryIsoCode):
        selection = "date, iso_code, new_cases"
        # loc = "United States"
        intList = []
        executeStr = f"""SELECT {selection} FROM "covid-world" WHERE "iso_code" LIKE '%{countryIsoCode}%' ORDER BY "date" """

        for i, row in enumerate(c.execute(executeStr)):
            if row[0] not in CoVID_Data.keys():  # List is empty
                dateList.append(row[0])
                d = {row[0]: {row[1]: int(row[2])}}
                CoVID_Data.update(d)
                # Below to generate intensity List, Not required
                intensityList[countryIsoCode].append(int(row[2]))

            else:
                d = {row[1]: int(row[2])}
                # print(d)
                CoVID_Data[row[0]].update(d)
                intensityList[countryIsoCode].append(int(row[2]))

    for countryISO in countryList:
        d = {countryISO: getMaxCountryCases(countryISO)}
        maxCountryCases.update(d)
        generateCoVIDData(countryISO)  # '2020-01-24': {'USA': 1, 'JPN': 0, 'ZAF': 0}

    # print("MaxCountryCases=", maxCountryCases)

    # selection = "date, iso_code, new_cases"
    # loc = "United States"
    #
    # executeStr = f"""SELECT {selection} FROM "covid-world" WHERE "iso_code" LIKE '%{loc}%' ORDER BY "date" """
    #
    # executeStr = r"""SELECT date, iso_code, new_cases FROM "main"."covid-world" WHERE "iso_code" LIKE '%JPN%'"""
    #
    # print(str(executeStr))
    # c.execute(executeStr)
    # # print(f'TotalRow = {len(c.fetchall())}')
    #
    # row = c.fetchone()
    # print("ROW=", row, "| Size =", len(row))
    #
    # for i in row:
    #     print(i)
    # print(type(row))

    # ('USA', '2020-01-23', '0')
    # selection = "date, iso_code, new_cases"

    # for i, row in enumerate(c.execute(executeStr)):
    #     infoDic = {"iso_code": row[1], "new_cases": row[2]}
    #     dateDic = {row[0]:infoDic}
    #     finalDic = {i}
    #     d = {i:row[1]: row[0], "date": row[0], "iso_code": row[0]}
    #     d2 = {}
    #     print(row)

    conn.close()


def addCountryXY_hapticSuit(name, x, y):
    newX = round(x / imageSize[0], 3)
    newY = round(y / imageSize[1], 3)

    d = {str(name): [newX, newY]}
    # print(f'{d} -- Type:{type(d)}')
    countryPathPointCoordDic.update(d)

def addCountryXY(name, x, y):
    d = {str(name): [x, y]}
    # print(f'{d} -- Type:{type(d)}')
    coord_Dic.update(d)

def runCV2():
    # import cv2
    # import numpy as np

    # Picture path
    img = cv2.imread(imageDir)
    coordXY_Tuple = []
    b = []

    def on_EVENT_LBUTTONDOWN(event, x, y, flags, param):
        if event == cv2.EVENT_LBUTTONDOWN:
            xy = "%d,%d" % (x, y)
            coordinate = (x, y)
            coordXY_Tuple.append(coordinate)
            # b.append(y)
            cv2.circle(img, (x, y), 1, (0, 0, 255), thickness=-1)
            cv2.putText(img, xy, (x, y), cv2.FONT_HERSHEY_PLAIN,
                        1.0, (0, 0, 0), thickness=1)
            cv2.imshow("image", img)
            print(x, y)

    cv2.putText(img, f"Pick Coordinate USA, South_Africa,and Japan in Order", tuple(imageSize), cv2.FONT_HERSHEY_SIMPLEX, 0.5,
                (0, 0, 0), thickness=1)
    cv2.namedWindow("image")
    cv2.setMouseCallback("image", on_EVENT_LBUTTONDOWN)
    cv2.imshow("image", img)
    cv2.waitKey(0)
    # print(a[0], b[0])
    # print(f'CV2-Coordinate:{a}')

    for i, country in enumerate(countryList):
        addCountryXY_hapticSuit(country, coordXY_Tuple[i][0], coordXY_Tuple[i][1])
        addCountryXY(country, coordXY_Tuple[i][0], coordXY_Tuple[i][1])


# for item in tempList:
#     print(item)
#     addCountry(item[0] ,item[1] ,item[2])

def genPathPoint(country, intensity, mode):
    x = countryPathPointCoordDic[country][0]
    y = countryPathPointCoordDic[country][1]
    # value = 0

    if mode in countryList:
        value = round(intensity * 100 / maxCountryCases[mode])
        if value > 100:
            value = 100
        d = {"x": x, "y": y, "intensity": value}
        return d
    elif mode == 0:
        value = round(intensity * 100 / maxCountryCases[country])

        d = {"x": x, "y": y, "intensity": value}
        return d
    else:
        print("Error: Use \"USA, JPN, ZAF, 0\"")


def main(mode):  # 0 = default(respective), "CountryISO" = setCountryMax as MaxIntensity
    global wait, millis, guiCircleModifier
    print('======================== DEBUG PRINT MAIN ========================')

    # print(f'Test[{len()}]: {}')
    runCV2()
    initializeCovidWorldData()
    print(f'coord_Dic ={coord_Dic}')
    print(f'countryPathPointCoordDic ={countryPathPointCoordDic}')
    # print(f'COVIDData[{len(CoVID_Data.keys())}]: {CoVID_Data}')
    print("MaxCountryCases=", maxCountryCases)

    # print(f'intensityList[{len(intensityList)}]: {intensityList}')

    print('======================== END DEBUG PRINT MAIN ========================')

    # wait = 0.2
    # millis = 200

    def test():
        pathPoint = [{"x": 0.5, "y": 0.5, "intensity": 100}]
        for i in range(5):
            player.submit_path("backFrame", "VestBack", pathPoint, millis)
            sleep(wait)

    # ================== Visualization CODE (matplotlib)===================
    def initGUIMatPlotlib():
        import matplotlib.pyplot as plt
        #matplotlib qt5
        fig, ax = plt.subplots(figsize=(6, 9))
        img = plt.imread(imageDir)
        ax.imshow(img)

    # ================== Main CODE Run===================  
    player = haptic_player.HapticPlayer()
    sleep(0.1)
    sleep(2)

    import matplotlib.pyplot as plt
    import matplotlib.animation as animation
    
    fig, ax = plt.subplots(figsize=(6, 9))
    img = plt.imread(imageDir)
    ax.imshow(img)
    
    plt.xlim([0, imageSize[0]])
    plt.ylim([0, imageSize[1]])

    test()  # For triggering BHaptic (prevent   data loss)

    modifier = 30
    color = {"USA":'tab:red', "ZAF":'tab:purple', "JPN":'tab:green'}

    # =============== Size Legend Display ===============
    # Dummy Variables:
    x = [0,0,0,0]
    y = [0,0,0,0]
    size_List = [25,50,75,100]
    lsize = []
    for i,value in enumerate(size_List):
        ax.scatter(x[i],y[i],s=(size_List[i]*modifier), label=f"{str(size_List[i])}%",facecolors='none',edgecolors='r',alpha=0.6)
    
    # scatter = ax.scatter(x,y,s=lsize, label=size_List)
    # handles, labels = ax.legend_elements(prop="sizes", alpha=0.6)
    # legend2 = ax.legend(handles, labels, loc="upper right", title="Sizes")
    title_lab = f"% of Highest \n Recorded Daily Cases"
    legend2 = ax.legend(loc="upper right", title=title_lab, handletextpad=2)
    # legend2._legend_box.align='right'
    # ax.add_artist(legend2)
    ax.cla()
    
    #=====================================================
    
    def haptic_thread(pathPoint,millis,wait):
            player.submit_path("backFrame", "VestBack", pathPoint, millis)
            sleep(wait)
    
    

    for i, (date, country_data) in enumerate(CoVID_Data.items()):
        # CoVID_Data Sample:
        # {'2020-01-23': {'USA': 0, 'ZAF': 0, 'JPN': 0},
        #   '2020-01-24': {'USA': 1, 'ZAF': 0, 'JPN': 0}, ...}

        # ===== Debug Skip =====
        if debugSkip != 0 and i <= debugSkip:
            continue

        # ===== Temporary Variables =====
        pathPoint = []              #For submit_Path(), BHaptic
        countryPercentDisplay = []  #Scale/Area of Scatterdot (%), no modifier
        caseDisplay = []            #Cases From each country
        ax.cla()                    #Clears subplot for new Draw

        for country, cases in country_data.items():
            pathPoint.append(genPathPoint(country, cases, mode))
            # For Text Display:
            if mode in countryList:
                percValue = round(cases / maxCountryCases[mode] * 100)
                if percValue > 100:
                    percValue = 100
            else:
                percValue = round(cases / maxCountryCases[country] * 100)

            countryPercentDisplay.append({country: percValue})
            # TODO: rewrite "caseDisplay" as dic{country:cases}
            caseDisplay.append(cases)
            ax.scatter(
                coord_Dic[country][0],  # Get X Coordinate of Country
                coord_Dic[country][1],  # Get Y Coordinate of Country
                s=percValue * guiCircleModifier,  # Area based on Max percent*(Modifier)
                c=color[country],
                label=country,
                alpha=0.6
            )

        global displayText
        newlist = [""]
        displayText = (f"#{i}| Date:{date}| data={country_data} | percentage={countryPercentDisplay} \n")
        print(displayText)

        # ===== GUI TEST ======
        ax.set_title(f"COVID New Cases Worldwide: Date: {date}",loc="left")
        ax.imshow(img)
        ax.grid(True)
        # TODO: rewrite "caseDisplay" as dic{country:cases}
        dataText = f"""
        New Cases:
        United States:  {caseDisplay[0]}
        South Africa:   {caseDisplay[1]}
        Japan:      {caseDisplay[2]}"""
        ax.text(100,(imageSize[1]-10),dataText,horizontalalignment="center")
        ax.add_artist(legend2)
        lgnd = ax.legend(loc="lower right", scatterpoints=1, fontsize=10)
        for i in range(len(countryList)):
            lgnd.legendHandles[i]._sizes = [100]
        # ===== GUI TEST ======
        t1 = threading.Thread(target=haptic_thread, args=(pathPoint,millis,wait,))
        t1.start()

        fig.canvas.draw()
        t1.join()
        # player.submit_path("backFrame", "VestBack", pathPoint, millis)
        # sleep(wait)        
        fig.canvas.flush_events()

    # window.close()


# ============ RUNTIME ==============
# main("JPN")
# main("USA")
main(0)

======================== DEBUG PRINT MAIN ========================
189 640
383 348
194 92
coord_Dic ={'USA': [189, 640], 'ZAF': [383, 348], 'JPN': [194, 92]}
countryPathPointCoordDic ={'USA': [0.344, 0.8], 'ZAF': [0.696, 0.435], 'JPN': [0.353, 0.115]}
MaxCountryCases= {'USA': 205557, 'ZAF': 13944, 'JPN': 2679}
======================== END DEBUG PRINT MAIN ========================
#301| Date:2020-11-19| data={'USA': 187963, 'ZAF': 2514, 'JPN': 2382} | percentage=[{'USA': 91}, {'ZAF': 18}, {'JPN': 89}] 

#302| Date:2020-11-20| data={'USA': 196010, 'ZAF': 3105, 'JPN': 2426} | percentage=[{'USA': 95}, {'ZAF': 22}, {'JPN': 91}] 

#303| Date:2020-11-21| data={'USA': 178073, 'ZAF': 2646, 'JPN': 2586} | percentage=[{'USA': 87}, {'ZAF': 19}, {'JPN': 97}] 

#304| Date:2020-11-22| data={'USA': 142734, 'ZAF': 2270, 'JPN': 2163} | percentage=[{'USA': 69}, {'ZAF': 16}, {'JPN': 81}] 

#305| Date:2020-11-23| data={'USA': 171462, 'ZAF': 2080, 'JPN': 1520} | percentage=[{'USA': 83}, {'ZAF': 15}, {'JPN':